<center><font size="6"><b>Комп'ютерний практикум 6.

<center><b> Алгоритми текстової класифікації </font>





<center><img src="https://miro.medium.com/v2/resize:fit:1100/format:webp/1*wE5wZtt-aR2mNnwUoMSEpg.jpeg" width="600"></center>

In [1]:
# Встановлюємо необхідні пакети
! pip install --upgrade pip
! pip install pyspark
! pip install findspark
! pip install pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.5 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: pip
    Found existing installation: pip 22.0.2
    Uninstalling pip-22.0.2:
      Successfully uninstalled pip-22.0.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 75.3 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840629 sha256=58ae4ab782834fa008f55f4bd38043e0629b338d3078e59f04c7024709118e08
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 MB 41.1 MB/s eta 0:00:0000:0100:01


In [2]:
# імпорт допоміжної бібліотеки для організації правильного шляху до Spark
import findspark
findspark.init()

In [2]:
! pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 23.2 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.3/16.3 MB 70.0 MB/s eta 0:00:0000:01


In [4]:
#імпорт бібліотек та модулів

import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [5]:
# Створення spark session

spark = SparkSession \
    .builder \
    .appName("My_Spark_Text_Сlassification_2") \
    .config("spark.executor.memory", "4g") \
    .config("spark.driver.memory", "4g")\
    .config("spark.executor.cores", "4") \
    .config("spark.sql.shuffle.partitions", "200") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1")\
    .config("spark.driver.maxResultSize", "4g") \
    .getOrCreate()

24/11/30 10:24:31 WARN Utils: Your hostname, ubuntu-server resolves to a loopback address: 127.0.1.1; using 192.168.50.183 instead (on interface eno1)
24/11/30 10:24:31 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/30 10:24:31 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [6]:
# Використаємо `read_csv` з pandas для завантаження датасету
df = pd.read_csv('data/Covid_text.csv', encoding='latin-1')

Датасет охоплює множину повідомлень Tweeter, що стосуються Covid-19 з різними настроями:
- Extremely Negative
- Negative
- Neutral
- Positive
- Extremely Positive

In [7]:
# Перегляд датасету
df.head(5)

,UserName,ScreenName,Location,TweetAt,OriginalTweet,Sentiment
0,1,44953,NYC,02-03-2020,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,2,44954,"Seattle, WA",02-03-2020,When I couldn't find hand sanitizer at Fred Me...,Positive
2,3,44955,NaN,02-03-2020,Find out how you can protect yourself and love...,Extremely Positive
3,4,44956,Chicagoland,02-03-2020,#Panic buying hits #NewYork City as anxious sh...,Negative
4,5,44957,"Melbourne, Victoria",03-03-2020,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [8]:
df.columns

Index(['UserName', 'ScreenName', 'Location', 'TweetAt', 'OriginalTweet',
       'Sentiment'],
      dtype='object')

In [9]:
# Оберемо два стовпчика з техстом та настороями (класами)
df= df[['OriginalTweet', 'Sentiment']]
df.head()

,OriginalTweet,Sentiment
0,TRENDING: New Yorkers encounter empty supermar...,Extremely Negative
1,When I couldn't find hand sanitizer at Fred Me...,Positive
2,Find out how you can protect yourself and love...,Extremely Positive
3,#Panic buying hits #NewYork City as anxious sh...,Negative
4,#toiletpaper #dunnypaper #coronavirus #coronav...,Neutral


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44955 entries, 0 to 44954
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   OriginalTweet  44955 non-null  object
 1   Sentiment      44955 non-null  object
dtypes: object(2)
memory usage: 702.5+ KB


In [11]:
#завантажимо дані в spark dataframe
tdf = spark.createDataFrame(df)

In [12]:
#виведемо dataframe на екран
tdf.show()

+--------------------+-------------------+
|       OriginalTweet|          Sentiment|
+--------------------+-------------------+
|TRENDING: New Yor...| Extremely Negative|
|When I couldn't f...|          Positive |
|Find out how you ...|Extremely Positive |
|#Panic buying hit...|           Negative|
|#toiletpaper #dun...|            Neutral|
|Do you remember t...|            Neutral|
|Voting in the age...|          Positive |
|@DrTedros "We can...|            Neutral|
|HI TWITTER! I am ...| Extremely Negative|
|Anyone been in a ...|Extremely Positive |
|Best quality couc...|          Positive |
|Beware of counter...| Extremely Negative|
|Panic food buying...| Extremely Negative|
|#Covid_19 Went to...|Extremely Positive |
|While we were bus...|          Positive |
|#AirSewa  @flyspi...| Extremely Negative|
|What Precautionar...|Extremely Positive |
|When youÂre stoc...|            Neutral|
|That's about a we...|          Positive |
|Studies show the ...|Extremely Positive |
+----------

## **Підготовка даних**
Підготовка даних до моделювання виконується  у декілька кроків (__Model Pipeline__)

У PySpark `Pipeline` — це об’єкт, що поєднує кілька етапів обробки.

- regexTokenizer: Токенізація
- stopwordsRemover: Видалення стоп-слів
- countVectors: Векторизація
- StringIndexer : кодування рядковий стовпчик міток у стовпчик індексів міток. Індекси знаходяться у діапазоні (0, numLabels), впорядковані за частотою міток, тому найчастіша мітка отримує індекс 0.
- Split Train/Test

### Переваги Model Pipeline:
- **Автоматизація та відтворюваність**: спрощує запуск задач, особливо при великих обсягах даних.
- **Масштабованість**: легкість інтеграції нових даних або оновлення існуючих етапів.
- **Управління ресурсами**: дозволяє ефективно розподіляти обчислювальні ресурси для різних етапів.


In [13]:
from pyspark.ml import Pipeline

In [14]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, OneHotEncoder, StringIndexer, VectorAssembler, HashingTF, IDF


In [15]:
tokenizer_tdf = Tokenizer(inputCol='OriginalTweet',outputCol='tokenized_Tweet')

In [16]:
add_stopwords = ['a','the','or', 'i', 'and', 'is', 'to','in', 'on']
stopwordsRemover_tdf = StopWordsRemover(inputCol='tokenized_Tweet', outputCol='stopwordsremoved_Tweet').setStopWords(add_stopwords)

In [17]:
countVectors_tdf = CountVectorizer(inputCol='stopwordsremoved_Tweet', outputCol='countvectorizer_Tweet')

In [18]:
label_stringIdx_tdf = StringIndexer(inputCol = "Sentiment", outputCol = "Sentiment_label")

In [19]:
pipeline = Pipeline(stages=[tokenizer_tdf, stopwordsRemover_tdf, countVectors_tdf, label_stringIdx_tdf])

pipelineFit = pipeline.fit(tdf)
dataset = pipelineFit.transform(tdf)
dataset.show(5)

+--------------------+-------------------+--------------------+----------------------+---------------------+---------------+
|       OriginalTweet|          Sentiment|     tokenized_Tweet|stopwordsremoved_Tweet|countvectorizer_Tweet|Sentiment_label|
+--------------------+-------------------+--------------------+----------------------+---------------------+---------------+
|TRENDING: New Yor...| Extremely Negative|[trending:, new, ...|  [trending:, new, ...| (128797,[15,18,33...|            4.0|
|When I couldn't f...|          Positive |[when, i, couldn'...|  [when, couldn't, ...| (128797,[0,1,2,3,...|            5.0|
|Find out how you ...|Extremely Positive |[find, out, how, ...|  [find, out, how, ...| (128797,[5,21,36,...|            3.0|
|#Panic buying hit...|           Negative|[#panic, buying, ...|  [#panic, buying, ...| (128797,[1,15,33,...|            1.0|
|#toiletpaper #dun...|            Neutral|[#toiletpaper, #d...|  [#toiletpaper, #d...| (128797,[1,4,29,3...|            2.0|


24/11/30 10:25:29 WARN DAGScheduler: Broadcasting large task binary with size 1915.1 KiB


In [20]:
# Train/Test Split
(trainingData, testData) = dataset.randomSplit([0.75, 0.25], seed = 623)
print("Кількість спостережень в тренувальній вибірці: " + str(trainingData.count()))
print("Кількість спостережень в тестовій вибірці: " + str(testData.count()))

24/11/30 10:25:36 WARN DAGScheduler: Broadcasting large task binary with size 1930.1 KiB


Кількість спостережень в тренувальній вибірці: 33529


24/11/30 10:25:36 WARN DAGScheduler: Broadcasting large task binary with size 1930.1 KiB


Кількість спостережень в тестовій вибірці: 11426


##__Логістична регресія (LR - Logistic Regression))__

Логістична регресія є частинним випадком лінійного бінарного класифікатора, яка повертає значення функції у межах від $0$ до $1$, що можна інтерпретувати як ймовірність $p_+$ того, що об'єкт $\vec{x_i}$ відноситься до классу $"+"$:


$$\large p_+ = P\left(y_i = 1 \mid \vec{x_i}, \vec{w}\right) $$


Прогнозування не просто відповіді, а саме ймовірності віднесення до класу $"+"$ в багатьох задачах є важливою бізнес-вимогою. Наприклад, в задачах кредитного скорингу, де застосовується  логістична регресія, часто прогнозують ймовірність неповернення кредиту ($p_+$). Клієнті сортують по цій передбачуваній ймовірності (за спаданням), і отримують скоркарту — по суті, рейтинг клієнтів від "ненадійних" до "надійних".

За допомогою ЛР ми хочемо спрогнозувати ймовірність $p_+ \in [0,1]$. Лінійна регресія за допомогою МНК дає можливість побудувати лінійний прогноз: $b(\vec{x}) = \vec{w}^T \vec{x} \in \mathbb{R}$.

Для того, щоб перетворити отримане значення в ймовірність необхідна функція $f: \mathbb{R} \rightarrow [0,1].$

В моделі логістичної регресії береться функція ___"сігмоїда" (логіт-функція)___: $$\sigma(z) = \frac{1}{1 + \exp^{-z}}$$.


<center><img src="https://static.javatpoint.com/tutorial/machine-learning/images/logistic-regression-in-machine-learning.png" width="400"></center>

Позначимо $P(X)$ ймовірність події $X$. Тоді відношення ймовірностей $OR(X)$ визначається з $\frac{P(X)}{1-P(X)}$ — відношення ймовірностей того, відбудеться подія чи ні. Ймовірність та відношення шансів містять однакову інформацію. Проте, $P(X)$ знаходится в межах від $0$ до $1$, а $OR(X)$ - в межах від $0$ до $\infty$.


Якщо прологарифмувати $OR(X)$, то $\log{OR(X)} \in \mathbb{R}$, його і будемо прогнозувати методом МНК.

Логістична регресія буде робити прогноз $p_+ = P\left(y_i = 1 \mid \vec{x_i}, \vec{w}\right)$.


__Крок 1.__ Обчислити значення $w_{0}+w_{1}x_1 + w_{2}x_2 + ... = \vec{w}^T\vec{x}$. (рівняння $\vec{w}^T\vec{x} = 0$ задає гіперплощину, яка ділить ознаки на 2 класи);


__Крок 2.__ Обчислюємо логарифм відношення шансів: $ \log(OR_{+}) = \vec{w}^T\vec{x}$.


__Крок 3.__ Обчислюємо $p_{+}$:

$$\large p_{+} = \frac{OR_{+}}{1 + OR_{+}} = \frac{\exp^{\vec{w}^T\vec{x}}}{1 + \exp^{\vec{w}^T\vec{x}}} = \frac{1}{1 + \exp^{-\vec{w}^T\vec{x}}} = \sigma(\vec{w}^T\vec{x})$$


В правій частині отримали сигмоїд-функцію.

Логістична регресія прогнозує ймовірність віднесення об'єкту до класу "+" як сигмоїд-перетворення лінійної комбінації вектора вагів моделі та вектору ознак об'єкту.

$$\large p_+(x_i) = P\left(y_i = 1 \mid \vec{x_i}, \vec{w}\right) = \sigma(\vec{w}^T\vec{x_i}). $$

<center><img
src="https://ibm.box.com/shared/static/kgv9alcghmjcv97op4d6onkyxevk23b1.png" width = "400" align = "center"></center>


###__Види логістичної регресії__

1. ___Бінарна або біноміальна___
> в такій класифікації цільова змінна має два можливих результати: $1$ та $0$.
2. ___Поліноміальна___
> в такій класифікації цільова змінна може мати 3 та більше можливих невпорядкованих результатів або результат, як категоріальну змінну.
3. ___Порядкова регресія___
> в такій класифікації цільова змінна може мати 3 та більше можливих впорядкованих результатів, які  мають кількісну інтерпретацію.

###__Передумови для логістичної регресії__

1. ЛР передбачає, що мультиколінеарність серед незалежних змінних мінімальна або відсутня.

2. ЛР передбачає, що незалежні змінні лінійно зв'язані з логарифмом шансів.

3. ЛР потребує великого розміру вибірки та правильного прогнозування.

4. ЛР передбачає, що спостереження незалежні.

### __Переваги методу__

1. Модель ЛР не тільки класифікує, а дає ймовірності належності класу.

2. ЛР дає не тільки міру того, на скільки релевантний предиктор (розмір коефіцієнту), але і "напрям" асоціації (додатній або від'ємний). ЛР простіше реалізувати, інтерпретувати та ефективно навчити.

3. ЛР є досить ефективною, коли набір даних має линійно відокремлювані функції.

4. В низьковимірному наборі даних, які мають достатню кількість об'єктів для навчання, ЛР не так схильна до перенавчання.

### __Недоліки методу__

1. ЛР перенавчається на багатовимірних наборах даних.

2. Низька якість для нелінійних функцій

3. Не в змозі працювати зі складними (комплексними) відношеннями між змінними, та змінними, які слабко пов'язані з цільовою змінною



**`LogisticRegression`** з модуля `pyspark.ml.classification` використовується для побудови моделі логістичної регресії в PySpark, що належить до Spark MLlib — бібліотеки для машинного навчання в Apache Spark.

### Основні параметри `LogisticRegression`:

- **`maxIter`**: максимальна кількість ітерацій для алгоритму оптимізації.
  - **Типове значення**: Зазвичай за замовчуванням значення `maxIter` складає `100`, хоча в деяких випадках (залежно від платформи чи реалізації) його може бути змінено.
  - **Велике значення**: Забезпечує точніше мінімізацію функції втрат, особливо якщо дані складні, але потребує більше часу на обчислення.
  - **Мале значення**: Зменшує час на навчання, проте може призвести до незбалансованої моделі (неповне навчання).

- **`regParam`**: параметр регуляризації для контролю надмірного навчання (перенавчання).Значення параметра **`regParam`** зазвичай лежить у межах від `0.0` до `1.0`, де:
  - **`regParam = 0.0`** означає відсутність регуляризації, тобто модель використовує тільки логістичну регресію без штрафів.
  - **Великі значення**, ближчі до `1.0`, означають сильнішу регуляризацію. Проте надто велике значення `regParam` може сильно знизити точність моделі, оскільки занадто обмежує її параметри.

- **`elasticNetParam`**: параметр Elastic Net, який дозволяє комбінувати $L_1$ і $L_2$-регуляризацію.
  - **`elasticNetParam = 0.0`**: Вказує на **$L_2$-регуляризацію** (Ridge-регуляризація). Тут модель намагається зменшити суму квадратів ваг, що добре працює для даних з корельованими ознаками.
  - **`elasticNetParam = 1.0`**: Використовується **$L_1$-регуляризація** (Lasso-регуляризація), яка сприяє розрідженості моделі, встановлюючи деякі ваги на нуль, тим самим видаляючи незначущі ознаки.
  - **Проміжні значення** (наприклад, `0.5`): Поєднують обидві регуляризації. Наприклад, `elasticNetParam = 0.5` означає рівний баланс між $L_1$ та $L_2$-регуляризацією.

- **`threshold`**: поріг для визначення класу в бінарній класифікації. Значення `threshold` визначає, при якій ймовірності модель присвоює спостереження до класу $«1»$ або $«0»$.
  - **Значення `threshold = 0.5`** (значення за замовчуванням): Якщо ймовірність належності спостереження до класу $«1»$ більша або дорівнює `0.5`, то модель класифікує його як клас $«1»$. Якщо менша за `0.5`, то — клас $«0»$.
  - **Зниження `threshold`** (наприклад, до `0.3`): Більше спостережень будуть класифіковані як клас $«1»$, що підвищує чутливість (Recall), але може знизити точність (Precision).
  - **Підвищення `threshold`** (наприклад, до `0.7`): Більше спостережень будуть класифіковані як клас $«0»$, що може підвищити точність, але знизити чутливість.

### Приклад використання `LogisticRegression` в PySpark:

```python
from pyspark.ml.classification import LogisticRegression

# Створення екземпляра LogisticRegression
lr = LogisticRegression(featuresCol="features", labelCol="label", maxIter=10, regParam=0.01)

# Навчання моделі
model = lr.fit(trainingData)

# Використання моделі для прогнозування
predictions = model.transform(testData)
```

In [21]:
from pyspark.ml.classification import LogisticRegression

In [22]:
lr = LogisticRegression(featuresCol="countvectorizer_Tweet", labelCol="Sentiment_label", maxIter=10, regParam=0.3, elasticNetParam=0)



In [23]:
lrModel = lr.fit(trainingData)
predictions_lr = lrModel.transform(testData)
predictions_lr.filter(predictions_lr['prediction'] == 0) \
    .select("OriginalTweet","Sentiment","Sentiment_label","probability","prediction") \
    .orderBy("probability", ascending=False) \
    .show(n = 10, truncate = 30)



24/11/30 10:26:26 WARN DAGScheduler: Broadcasting large task binary with size 1961.7 KiB
24/11/30 10:26:27 WARN DAGScheduler: Broadcasting large task binary with size 1962.8 KiB
24/11/30 10:26:28 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/11/30 10:26:28 WARN DAGScheduler: Broadcasting large task binary with size 1962.2 KiB
24/11/30 10:26:28 WARN DAGScheduler: Broadcasting large task binary with size 1963.4 KiB
24/11/30 10:26:29 WARN DAGScheduler: Broadcasting large task binary with size 1962.2 KiB
24/11/30 10:26:29 WARN DAGScheduler: Broadcasting large task binary with size 1963.4 KiB
24/11/30 10:26:29 WARN DAGScheduler: Broadcasting large task binary with size 1962.2 KiB
24/11/30 10:26:29 WARN DAGScheduler: Broadcasting large task binary with size 1963.4 KiB
24/11/30 10:26:29 WARN DAGScheduler: Broadcasting large task binary with size 1962.2 KiB
24/11/30 10:26:29 WARN DAGScheduler: Broadcasting large task binary with size 1963.4 KiB
24/

+------------------------------+-------------------+---------------+------------------------------+----------+
|                 OriginalTweet|          Sentiment|Sentiment_label|                   probability|prediction|
+------------------------------+-------------------+---------------+------------------------------+----------+
| Sanitizer Italy Wuhan Groc...|           Negative|            1.0|[0.9247152641761016,0.02876...|       0.0|
|Surprising Rod Sims doesnÂ...|Extremely Positive |            3.0|[0.9246714416494948,0.01869...|       0.0|
|ÂLock DownÂ a #poem by Po...|           Negative|            1.0|[0.9184317447913909,0.02744...|       0.0|
|The FDA Has Recently: -Refu...|Extremely Positive |            3.0|[0.8928734643172614,0.01177...|       0.0|
|In times of crisis like 19 ...|           Positive|            0.0|[0.8826941811154077,0.03103...|       0.0|
|No toxic chemicals. ? Use g...|           Positive|            0.0|[0.878321827630632,0.029344...|       0.0|
|

## **Точність моделі**
`MulticlassClassificationEvaluator` — це клас у PySpark, який використовується для оцінювання моделей багатокласової класифікації за різними метриками. Він допомагає визначити продуктивність моделей, які передбачають більше двох класів.

### Основні метрики, що підтримуються:
1. **Accuracy** (Точність): Частка правильних передбачень серед усіх передбачень.
2. **Weighted Precision** (Зважена точність): Середньозважена точність для всіх класів.
3. **Weighted Recall** (Зважена повнота): Середньозважена повнота для всіх класів.
4. **F1 Score** (Зважена F1-оцінка): Середньозважене гармонійне середнє між точністю та повнотою.

### Основні параметри:
- **`labelCol`**: Назва колонки, яка містить істинні мітки класів.
- **`predictionCol`**: Назва колонки з передбаченими мітками.
- **`metricName`**: Назва метрики, яку потрібно обчислити. Може бути одним з:
  - `"accuracy"`
  - `"weightedPrecision"`
  - `"weightedRecall"`
  - `"f1"`

### Приклад використання:
```python
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Створення об'єкта оцінювача
evaluator = MulticlassClassificationEvaluator(
    labelCol="label",              # Істинні мітки класів
    predictionCol="prediction",    # Передбачені мітки класів
    metricName="accuracy"          # Метрика для оцінки
)

# Оцінка моделі
accuracy = evaluator.evaluate(predictions)
print(f"Точність моделі: {accuracy:.2f}")
```

### Пояснення:
- **`evaluator.evaluate(predictions)`**: Виконує оцінку моделі на основі передбачень, переданих у `predictions` (DataFrame з передбаченими значеннями та істинними мітками).
- **`metricName`** визначає метрику, яку потрібно обчислити. У прикладі вище обрана метрика — точність (`accuracy`).

### Переваги:
- **Гнучкість**: Підтримує різні метрики, що дозволяє вибрати ту, яка найкраще відповідає вашим цілям.
- **Простота використання**: Легко інтегрується з об'єктами моделей і DataFrame у PySpark.
- **Зважені метрики**: Враховують нерівномірний розподіл класів, що робить оцінку більш точною для незбалансованих даних.



In [24]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [25]:
evaluator_lr_ac = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction")
print("Точність моделі логістичної регресії : ", evaluator_lr_ac.evaluate(predictions_lr))

24/11/30 10:26:55 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Точність моделі логістичної регресії :  0.4343601107305304


In [26]:
evaluator_lr_wp = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction", metricName="weightedPrecision" )
print("Зважена точність моделі логістичної регресії : ", evaluator_lr_wp.evaluate(predictions_lr))

24/11/30 10:26:57 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Зважена точність моделі логістичної регресії :  0.4695766970823562


In [27]:
evaluator_lr_wr = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction", metricName="weightedRecall" )
print("Зважена повнота моделі логістичної регресії : ", evaluator_lr_wr.evaluate(predictions_lr))

24/11/30 10:26:59 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Зважена повнота моделі логістичної регресії :  0.43759845965342203


In [28]:
evaluator_lr_f1 = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction", metricName="f1" )
print("Зважена F1-оцінка моделі логістичної регресії : ", evaluator_lr_f1.evaluate(predictions_lr))

24/11/30 10:27:01 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Зважена F1-оцінка моделі логістичної регресії :  0.4343601107305304


### **CrossValidator**

<center><img
src="https://preview.redd.it/doubt-in-k-fold-cross-validation-v0-rn7ev6upxd6a1.png?width=1200&format=png&auto=webp&s=38f46854f418150a899d9193b4b90e9f80a23409" width = "600" align = "center"></center>



`CrossValidator` — це клас у PySpark, який використовується для автоматичного налаштування гіперпараметрів моделі за допомогою крос-валідації. Він допомагає вибрати найкращу комбінацію параметрів, яка максимізує (або мінімізує) задану метрику оцінки на основі певної кількості фолдів (розділень даних).

1. **Розбиття даних**: `CrossValidator` розбиває навчальні дані на кілька фолдів (наприклад, 5 або 10 частин). Кожна частина по черзі використовується як тестовий набір, а решта — як навчальний.
2. **Перевірка кожної комбінації параметрів**: модель навчається для кожної комбінації параметрів з сітки на кожному фолді і обчислюється середнє значення метрики оцінювання.
3. **Вибір найкращої моделі**: Після прогону всіх комбінацій `CrossValidator` вибирає модель з найкращими результатами.

### Основні параметри `CrossValidator`:
- **`estimator`**: Модель, яку потрібно тренувати.
- **`estimatorParamMaps`**: Сітка параметрів, створена за допомогою `ParamGridBuilder()`.
- **`evaluator`**: Об'єкт для оцінювання моделей, який визначає метрику (наприклад, точність, F1, RMSE).
- **`numFolds`**: Кількість фолдів для крос-валідації. За замовчуванням використовується значення 3.

### Приклад використання:
```python
from pyspark.ml.tuning import CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator

# Підготовка об'єкта оцінювача
evaluator = BinaryClassificationEvaluator(metricName="areaUnderROC")

# Створення об'єкта CrossValidator
cv = CrossValidator(estimator=lr,                # Модель логістичної регресії
                    estimatorParamMaps=paramGrid, # Сітка параметрів
                    evaluator=evaluator,          # Оцінювач для обчислення AUC
                    numFolds=5)                  # 5-фолдова крос-валідація

# Тренування моделі з крос-валідацією
cvModel = cv.fit(trainingData)

# Використання найкращої моделі для передбачення на тестових даних
predictions = cvModel.transform(testData)
```

### Переваги використання `CrossValidator`:
- **Автоматизація підбору параметрів**: Знаходить оптимальні гіперпараметри для моделі.
- **Надійність оцінки**: Зменшує ймовірність перенавчання, використовуючи середнє значення метрики з кількох фолдів.
- **Гнучкість**: Підходить для різних моделей і оцінювачів.

### Недоліки:
- **Витрати часу**: Крос-валідація з великою кількістю фолдів і комбінацій параметрів може бути обчислювально довгою.
- **Складність для великих наборів даних**: Потрібна оптимізація для швидкої обробки великих обсягів даних.


### **Сітка гіперпараметрів**

__Сітка гіперпараметрів__ — це набір різних комбінацій значень параметрів, які модель тренування випробовує під час процесу налаштування. Метою є знайти комбінацію, яка максимізує метрику оцінювання моделі (наприклад, точність, F1-міру тощо).

`ParamGridBuilder()` — це клас у PySpark, який використовується для створення сітки гіперпараметрів для підбору моделі за допомогою методів крос-валідації або інших підходів оцінювання. Він дозволяє випробувати різні комбінації значень параметрів моделі, щоб знайти оптимальні для навчання.
- **`.addGrid()`**: Додає параметр моделі та список значень, які потрібно випробувати.
- **`.build()`**: Створює сітку параметрів для використання під час крос-валідації або іншої процедури пошуку.

### Приклад використання:
```python
from pyspark.ml.tuning import ParamGridBuilder

paramGrid = (ParamGridBuilder()
             .addGrid(model.param1, [0.1, 0.2, 0.3])  # Додати значення для param1
             .addGrid(model.param2, [5, 10, 15])      # Додати значення для param2
             .build())                                # Створити сітку
```

### Переваги використання:
- **Автоматизація підбору параметрів**: Спрощує процес пошуку оптимальних значень параметрів для підвищення продуктивності моделі.
- **Гнучкість**: Можна легко додати декілька параметрів для дослідження.
- **Ефективність**: Використовується разом з крос-валідацією (`CrossValidator`), щоб перевірити всі комбінації на різних підмножинах даних.

In [29]:
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [30]:
# Створення ParamGrid для Cross Validation
paramGrid = (ParamGridBuilder()
             .addGrid(lr.regParam, [0.1, 0.3]) # параметри регуляризації
             .addGrid(lr.elasticNetParam, [0.0, 0.1]) # Параметри Elastic Net (Ridge = 0)
#            .addGrid(model.maxIter, [10, 20, 50]) # Кількість ітерацій
             .build())

# Створення 3-fold CrossValidator
cv_lr = CrossValidator(estimator=lr, \
                    estimatorParamMaps=paramGrid, \
                    evaluator=evaluator_lr_ac, \
                    numFolds=3)
cvModel_lr = cv_lr.fit(trainingData)

predictions_cv_lr = cvModel_lr.transform(testData)

24/11/30 10:27:23 WARN DAGScheduler: Broadcasting large task binary with size 1940.0 KiB
24/11/30 10:27:24 WARN DAGScheduler: Broadcasting large task binary with size 1974.5 KiB
24/11/30 10:27:24 WARN DAGScheduler: Broadcasting large task binary with size 1975.6 KiB
24/11/30 10:27:24 WARN DAGScheduler: Broadcasting large task binary with size 1975.0 KiB
24/11/30 10:27:24 WARN DAGScheduler: Broadcasting large task binary with size 1976.2 KiB
24/11/30 10:27:24 WARN DAGScheduler: Broadcasting large task binary with size 1975.0 KiB
24/11/30 10:27:24 WARN DAGScheduler: Broadcasting large task binary with size 1976.2 KiB
24/11/30 10:27:25 WARN DAGScheduler: Broadcasting large task binary with size 1975.0 KiB
24/11/30 10:27:25 WARN DAGScheduler: Broadcasting large task binary with size 1976.2 KiB
24/11/30 10:27:25 WARN DAGScheduler: Broadcasting large task binary with size 1975.0 KiB
24/11/30 10:27:25 WARN DAGScheduler: Broadcasting large task binary with size 1976.2 KiB
24/11/30 10:27:25 WAR

In [31]:
# Найкраща модель
best_model_lr = cvModel_lr.bestModel

# Вивід оптимальних параметрів
optimal_reg_param = best_model_lr._java_obj.getRegParam()
optimal_elastic_net_param = best_model_lr._java_obj.getElasticNetParam()

print(f"Оптимальний regParam: {optimal_reg_param}")
print(f"Оптимальний elasticNetParam: {optimal_elastic_net_param}")

# Вивід оптимальної метрики
optimal_metric_lr = max(cvModel_lr.avgMetrics)
print(f"Оптимальна метрика (оцінена крос-валідацією): {optimal_metric_lr}")


Оптимальний regParam: 0.1
Оптимальний elasticNetParam: 0.1
Оптимальна метрика (оцінена крос-валідацією): 0.4628147339448953


In [32]:
# Оцінка найкращої моделі
evaluator = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction")
print("Test-set Accuracy is : ", evaluator.evaluate(predictions_cv_lr))

24/11/30 10:29:37 WARN DAGScheduler: Broadcasting large task binary with size 1968.5 KiB


Test-set Accuracy is :  0.4611696726550269


## **Наївний Байєс (Naive Bayes)**

**Наївний Байєс (Naive Bayes)** — це простий, але ефективний алгоритм класифікації, який базується на **Теоремі Байєса**. Він використовується для задач класифікації, особливо популярний у задачах обробки тексту, таких як класифікація електронної пошти (спам чи не спам), аналіз тональності тексту, класифікація документів тощо.

<center><img src="https://databasecamp.de/wp-content/uploads/naive-bayes-overview.png" width="400"></center>

Основна ідея алгоритму — це обчислення ймовірності того, що даний об'єкт належить до певного класу, на основі його характеристик. Назва "наївний" вказує на основне припущення алгоритму — незалежність характеристик. Алгоритм вважає, що кожна характеристика (наприклад, кожне слово у тексті) незалежно впливає на ймовірність того, що об'єкт належить до певного класу. Це припущення часто не є істинним на практиці, але дає добрі результати на багатьох задачах.

### Теорема Байєса

Наївний Байєс базується на Теоремі Байєса:

$$
P(A|B) = \frac{P(B|A) \cdot P(A)}{P(B)}
$$

де:
- $ P(A|B) $ — умовна ймовірність події $ A $ за умови події $ B $;
- $ P(B|A) $ — умовна ймовірність події $ B $ за умови події $ A $;
- $ P(A) $ та $ P(B)$ — апріорні ймовірності подій $ A $ та $ B $.

У контексті класифікації:
- $ P(C|X) $ — ймовірність того, що об'єкт з характеристиками $X $ належить класу $C$.
- $P(X|C) $ — ймовірність появи характеристик $ X $ за умови, що об'єкт належить класу $C$.
- $P(C)$ — апріорна ймовірність класу $C$.
- $P(X)$ — апріорна ймовірність характеристик $X$.

### Етапи алгоритму

1. **Обчислення апріорних ймовірностей класів** — на основі частоти кожного класу в навчальній вибірці обчислюється $P(C)$ для кожного класу $C$.

2. **Оцінка ймовірностей характеристик** — для кожного класу $C$ і кожної характеристики (наприклад, для кожного слова в тексті) обчислюється умовна ймовірність $P(X_i|C)$.

3. **Обчислення умовної ймовірності для нового об'єкта** — для нового об'єкта з характеристиками $X = \{X_1, X_2, \dots, X_n\} $ обчислюється умовна ймовірність для кожного класу $ C$ за формулою:

   $$
   P(C|X) \propto P(C) \cdot P(X_1|C) \cdot P(X_2|C) \cdots P(X_n|C)
   $$

   Зазвичай використовують логарифм ймовірностей, щоб уникнути проблем з переповненням чисел при множенні малих ймовірностей:
   
   $$
   \log P(C|X) = \log P(C) + \sum_{i=1}^n \log P(X_i|C)
   $$

4. **Призначення класу** — об'єкт належить тому класу, для якого обчислена ймовірність $ P(C|X) $ є максимальною.

### Види алгоритму

<center><img src="https://media.licdn.com/dms/image/v2/D4D12AQGtyG1s9QTyLw/article-cover_image-shrink_720_1280/article-cover_image-shrink_720_1280/0/1693914039946?e=2147483647&v=beta&t=g_CKWELZBn1adDuEvRJ951biLzeGuQ6u1Y0pc4Wknsg" width="600"></center>

Є кілька типів Наївного Байєса, залежно від розподілу ймовірностей, які використовуються для оцінки характеристик:

- **Multinomial Naive Bayes** — використовується для обчислення частоти появи характеристик (підходить для тексту, де характеристики — це слова).
- **Gaussian Naive Bayes** — припускає нормальний розподіл характеристик (застосовується для неперервних числових даних).
- **Bernoulli Naive Bayes** — працює з двійковими ознаками (наприклад, якщо характеристика є чи її немає).

### Переваги алгоритму

1. **Простота та швидкість** — алгоритм дуже простий у реалізації та швидкий у навчанні на великих наборах даних.
2. **Ефективний для текстової класифікації** — алгоритм добре працює з текстовими даними, де припущення незалежності характеристик є частково обґрунтованим.
3. **Мало ресурсозатратний** — не потребує великих обчислювальних ресурсів, оскільки потребує лише обчислення ймовірностей.
4. **Стійкість до шуму** — навіть за наявності зайвих або "шумних" ознак, може показувати хороші результати.

### Недоліки алгоритму

1. **Припущення незалежності** — головний недолік алгоритму, оскільки у реальних даних характеристики часто залежать одна від одної, що може знижувати точність моделі.
2. **Проблеми з нульовими ймовірностями** — якщо характеристика не зустрічалась у класі, то умовна ймовірність для цього класу стає нульовою, що ускладнює обчислення (вирішується через "додавання Лапласа").
3. **Не підходить для складних моделей** — алгоритм менш гнучкий для задач, де класифікація залежить від взаємозв’язків між ознаками.



**`NaiveBayes`** — це алгоритм машинного навчання, який використовується для класифікаційних задач. Він базується на використанні теореми Байєса з припущенням, що всі характеристики незалежні одна від одної. У PySpark є реалізація `NaiveBayes` у бібліотеці `pyspark.ml.classification`, що дозволяє застосовувати цей алгоритм для аналізу великих обсягів даних.

### Основні параметри `NaiveBayes`:
- **`smoothing`**: Параметр згладжування Лапласа для обробки рідкісних випадків (за замовчуванням — 1.0).
- **`modelType`**: Тип моделі (наприклад, `"multinomial"` для багатономінальних розподілів або `"bernoulli"` для бінарних даних).
- **`featuresCol` і `labelCol`**: Назви колонок для ознак і міток у DataFrame.

### Приклад використання:
```python
from pyspark.ml.classification import NaiveBayes

# Ініціалізація моделі NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", labelCol="label", featuresCol="features")

# Тренування моделі
model = nb.fit(trainingData)

# Передбачення на тестових даних
predictions = model.transform(testData)

```


In [33]:
from pyspark.ml.classification import NaiveBayes

In [34]:
# Навчаємо модель NaiveBayes
nb = NaiveBayes(smoothing=1.0, modelType="multinomial", featuresCol="countvectorizer_Tweet", labelCol="Sentiment_label")

nbModel = nb.fit(trainingData)
prediction_nb = nbModel.transform(testData)
prediction_nb.select("OriginalTweet","Sentiment","probability","Sentiment_label","prediction").show(n = 10, truncate = 30)

24/11/30 10:30:05 WARN DAGScheduler: Broadcasting large task binary with size 1957.8 KiB
24/11/30 10:30:05 WARN DAGScheduler: Broadcasting large task binary with size 1936.6 KiB


+------------------------------+-------------------+------------------------------+---------------+----------+
|                 OriginalTweet|          Sentiment|                   probability|Sentiment_label|prediction|
+------------------------------+-------------------+------------------------------+---------------+----------+
|"First time IÂve lived thr...| Extremely Negative|[2.2417745179141362E-27,0.9...|            4.0|       1.0|
|"ItÂs really working out. ...|Extremely Positive |[0.17922438260218804,0.6282...|            3.0|       1.0|
|"Losing free food is a blow...|          Positive |[0.59405634563169,0.3793734...|            5.0|       0.0|
|"We are doing everything we...|          Positive |[1.7628142347149743E-4,0.00...|            5.0|       3.0|
|#AirSewa  @flyspicejet is n...| Extremely Negative|[0.008298609262924247,0.984...|            4.0|       1.0|
|#BreakingNews #terrorists h...| Extremely Negative|[0.0034485448450175447,0.99...|            4.0|       1.0|
|

24/11/30 10:30:05 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


In [35]:
evaluator_nb_ac = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction")
print("Точність моделі Наївний Байес: ", evaluator_nb_ac.evaluate(prediction_nb))


24/11/30 10:30:50 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Точність моделі Наївний Байес:  0.4017092266509798


In [36]:
evaluator_nb_wp = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction", metricName="weightedPrecision" )
print("Зважена точність моделі Наївний Байес: ", evaluator_nb_wp.evaluate(prediction_nb))

24/11/30 10:30:52 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Зважена точність моделі Наївний Байес:  0.5008746560392439


In [37]:
evaluator_nb_wr = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction", metricName="weightedRecall" )
print("Зважена повнота моделі Наївний Байес: ", evaluator_nb_wr.evaluate(prediction_nb))

24/11/30 10:30:54 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Зважена повнота моделі Наївний Байес:  0.42368282863644324


In [38]:
evaluator_nb_f1 = MulticlassClassificationEvaluator(labelCol="Sentiment_label", predictionCol="prediction", metricName="f1" )
print("Зважена F1-оцінка моделі Наївний Байес: ", evaluator_nb_f1.evaluate(prediction_nb))

24/11/30 10:30:57 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Зважена F1-оцінка моделі Наївний Байес:  0.4017092266509798


In [39]:
# Налаштування сітки параметрів
paramGrid_nb = (ParamGridBuilder()
             .addGrid(nb.smoothing, [0.5, 1.0, 1.5])
        #    .addGrid(nb.modelType, ["multinomial", "bernoulli"])
             .build())

In [40]:
cv_nb = CrossValidator(estimator=nb,
                    estimatorParamMaps=paramGrid_nb,
                    evaluator=evaluator_nb_ac,
                    numFolds=3)

In [41]:
# Навчання моделі
cvModel_nb   = cv_nb.fit(trainingData)

24/11/30 10:31:04 WARN DAGScheduler: Broadcasting large task binary with size 1940.0 KiB
24/11/30 10:31:05 WARN DAGScheduler: Broadcasting large task binary with size 1970.5 KiB
24/11/30 10:31:05 WARN DAGScheduler: Broadcasting large task binary with size 1971.5 KiB
24/11/30 10:31:05 WARN DAGScheduler: Broadcasting large task binary with size 1939.9 KiB
24/11/30 10:31:05 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
24/11/30 10:31:06 WARN DAGScheduler: Broadcasting large task binary with size 1970.5 KiB
24/11/30 10:31:06 WARN DAGScheduler: Broadcasting large task binary with size 1971.5 KiB
24/11/30 10:31:06 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
24/11/30 10:31:06 WARN DAGScheduler: Broadcasting large task binary with size 1970.5 KiB
24/11/30 10:31:06 WARN DAGScheduler: Broadcasting large task binary with size 1971.5 KiB
24/11/30 10:31:07 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB
24/11/30 10:31:07 WARN DAGSche

In [42]:
# Прогнози на тестових даних
predictions_nb_cv = cvModel_nb.transform(testData)

In [43]:
# Оцінка
accuracy = evaluator_nb_ac.evaluate(predictions_nb_cv)
print(f"Точність на тестових даних: {accuracy}")

24/11/30 10:31:51 WARN DAGScheduler: Broadcasting large task binary with size 8.8 MiB


Точність на тестових даних: 0.44640657445728


In [44]:
# Найкраща модель
best_model_nb = cvModel_nb.bestModel

# Отримання оптимальних параметрів за допомогою extractParamMap()
best_params_nb = best_model_nb.extractParamMap()

# Виведення оптимального значення smoothing
best_smoothing = best_params_nb[best_model_nb.smoothing]
print(f"Оптимальне значення smoothing: {best_smoothing}")

# Виведення оптимального типу моделі
best_model_type = best_params_nb[best_model_nb.modelType]
print(f"Оптимальний тип моделі: {best_model_type}")


Оптимальне значення smoothing: 0.5
Оптимальний тип моделі: multinomial


##__Самостійні завдання__

> Скопіювати блок самостійних завдань в окремий файл ***LastName_CP6.ipynb***



### Завдання №1

1. Інсталювати та імпортувати необхідні бібліотеки та модулі
2. Створити Spark Session








In [1]:
# МІСЦЕ ДЛЯ КОДУ
import findspark
findspark.init()

import pandas as pd
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("My_Spark_Text_Сlassification_2") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "16g") \
    .config("spark.executor.cores", "8") \
    .config("spark.sql.shuffle.partitions", "100") \
    .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
    .config("spark.sql.autoBroadcastJoinThreshold", "-1") \
    .config("spark.driver.maxResultSize", "8g") \
    .getOrCreate()

24/12/01 17:58:36 WARN Utils: Your hostname, ubuntu-server resolves to a loopback address: 127.0.1.1; using 192.168.50.183 instead (on interface eno1)
24/12/01 17:58:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/01 17:58:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Завдання №2

* Завантажте фійл `text_data_med.csv` в локальне сховище
та застосуйте команду для зчитування файлу
`df = pd.read_csv('text_data_med.csv')`

Набір даних включає документи про рак, які класифікуються за 3 категоріями: «Рак щитовидної залози», «Рак товстої кишки», «Рак легенів».
Загальна кількість публікацій = 7569. у наборі даних є 3 мітки класів.

* перетворіть датасет у DataFrame PySpark
* перевірте на пропущені дані, при необхідності вилучіть порожні записи


In [2]:
# Load the CSV file into a Pandas DataFrame
df = pd.read_csv('data/text_data_med.csv', encoding='ISO-8859-1')

# Convert the Pandas DataFrame to a PySpark DataFrame
pyspark_df = spark.createDataFrame(df)

# Check for missing data and filter out rows with any null values
cleaned_pyspark_df = pyspark_df.na.drop()
# Display the cleaned PySpark DataFrame schema and a sample
cleaned_pyspark_df.printSchema()
cleaned_pyspark_df.show(10)

root
 |-- Unnamed: 0: long (nullable = true)
 |-- 0: string (nullable = true)
 |-- a: string (nullable = true)



24/12/01 17:58:42 WARN TaskSetManager: Stage 0 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.


+----------+--------------+--------------------+
|Unnamed: 0|             0|                   a|
+----------+--------------+--------------------+
|         0|Thyroid_Cancer|Thyroid surgery i...|
|         1|Thyroid_Cancer|" The adopted str...|
|         2|Thyroid_Cancer|coronary arteryby...|
|         3|Thyroid_Cancer| Solitary plasmac...|
|         4|Thyroid_Cancer| This study aimed...|
|         5|Thyroid_Cancer| This study was p...|
|         6|Thyroid_Cancer| This study was p...|
|         7|Thyroid_Cancer|Journal of Intern...|
|         8|Thyroid_Cancer| Gastric cancer G...|
|         9|Thyroid_Cancer| Scars Burns  Hea...|
+----------+--------------+--------------------+
only showing top 10 rows



### Завдання №3
Виконайте:
* токенізацію
* вилучення стоп слів по замовчуванню та задопомогою створення власного списку стоп слів, щоб було видно видалення слів при виводі на екран
* перетворення у векторну форму
* використайте об'єднання цих команд `pipeline`


In [3]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer

# Tokenization
tokenizer = Tokenizer(inputCol="a", outputCol="tokens")

# Default stop words removal
default_stopword_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens_default")

# Custom stop words removal
custom_stop_words = ['a', 'the', 'or', 'i', 'and', 'is', 'to', 'in', 'on']
custom_stopword_remover = StopWordsRemover(inputCol="filtered_tokens_default", outputCol="filtered_tokens_custom", stopWords=custom_stop_words)

# Vectorization
vectorizer = CountVectorizer(inputCol="filtered_tokens_custom", outputCol="features")

# Create a pipeline
pipeline = Pipeline(stages=[tokenizer, default_stopword_remover, custom_stopword_remover, vectorizer])

# Fit and transform the data
pipeline_model = pipeline.fit(cleaned_pyspark_df)
transformed_df = pipeline_model.transform(cleaned_pyspark_df)

# Show results
transformed_df.select("a", "tokens", "filtered_tokens_default", "filtered_tokens_custom", "features").show(10, truncate=False)

24/12/01 17:58:46 WARN TaskSetManager: Stage 1 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 17:58:50 WARN DAGScheduler: Broadcasting large task binary with size 3.2 MiB
24/12/01 17:58:50 WARN TaskSetManager: Stage 5 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.


+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

### Завдання №4
* побудуйте 3 моделі `Logistic Regression` з різними параметрами
* зробіть оцінку моделей
* налаштуйте параметри `CrossValidator` для моделі `Logistic Regression` та знайдіть оптимальні параметри моделі


In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Reduce the feature dimensionality using CountVectorizer
from pyspark.ml.feature import CountVectorizer

# Rename or drop the existing "features" column to avoid conflicts
transformed_df = transformed_df.drop("features")

vectorizer = CountVectorizer(inputCol="filtered_tokens_custom", outputCol="features", vocabSize=1000)
transformed_df = vectorizer.fit(transformed_df).transform(transformed_df)

# Split the sampled data into training and testing datasets
trainingData, testData = transformed_df.randomSplit([0.8, 0.2], seed=42)

# Define three Logistic Regression models with different parameters
lr1 = LogisticRegression(featuresCol="features", labelCol="Unnamed: 0", maxIter=5, regParam=0.3, elasticNetParam=0.0)
lr2 = LogisticRegression(featuresCol="features", labelCol="Unnamed: 0", maxIter=10, regParam=0.2, elasticNetParam=0.1)
lr3 = LogisticRegression(featuresCol="features", labelCol="Unnamed: 0", maxIter=15, regParam=0.1, elasticNetParam=0.2)

# Train the models
lr1_model = lr1.fit(trainingData)
lr2_model = lr2.fit(trainingData)
lr3_model = lr3.fit(trainingData)

# Evaluate the models on test data
evaluator = MulticlassClassificationEvaluator(labelCol="Unnamed: 0", predictionCol="prediction", metricName="accuracy")

accuracy_lr1 = evaluator.evaluate(lr1_model.transform(testData))
accuracy_lr2 = evaluator.evaluate(lr2_model.transform(testData))
accuracy_lr3 = evaluator.evaluate(lr3_model.transform(testData))

print(f"Accuracy of LR Model 1: {accuracy_lr1}")
print(f"Accuracy of LR Model 2: {accuracy_lr2}")
print(f"Accuracy of LR Model 3: {accuracy_lr3}")

# Define a base Logistic Regression model
lr = LogisticRegression(featuresCol="features", labelCol="Unnamed: 0")

# Create a parameter grid for CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.1, 0.3]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.1, 0.5]) \
    .build()

# Create a CrossValidator with fewer folds to reduce memory consumption
cv = CrossValidator(estimator=lr,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=2)  # Reduce number of folds

# Train the model using CrossValidator
cv_model = cv.fit(trainingData)

# Get the best model
best_model = cv_model.bestModel

# Evaluate the best model on the test set
best_model_predictions = best_model.transform(testData)
best_model_accuracy = evaluator.evaluate(best_model_predictions)

# Output the optimal parameters and metrics
optimal_reg_param = best_model._java_obj.getRegParam()
optimal_elastic_net_param = best_model._java_obj.getElasticNetParam()

print(f"Optimal regParam: {optimal_reg_param}")
print(f"Optimal elasticNetParam: {optimal_elastic_net_param}")
print(f"Test-set Accuracy of Best Model: {best_model_accuracy}")

24/12/01 17:58:53 WARN TaskSetManager: Stage 6 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 17:58:55 WARN TaskSetManager: Stage 10 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 17:58:57 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
24/12/01 17:58:57 WARN TaskSetManager: Stage 12 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 17:59:07 WARN TaskSetManager: Stage 14 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 17:59:23 WARN TaskSetManager: Stage 16 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 17:59:30 WARN TaskSetManager: Stage 18 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 17:59:37 WARN TaskSetManage

Accuracy of LR Model 1: 0.0
Accuracy of LR Model 2: 0.0
Accuracy of LR Model 3: 0.0


24/12/01 18:16:06 WARN TaskSetManager: Stage 233 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:16:06 WARN TaskSetManager: Stage 235 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:16:10 WARN TaskSetManager: Stage 237 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:16:14 WARN TaskSetManager: Stage 239 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:16:20 WARN TaskSetManager: Stage 241 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:16:31 WARN TaskSetManager: Stage 243 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:16:37 WARN TaskSetManager: Stage 245 contains a task of very large size (13873 KiB). The maximum recommended task size is 10

KeyboardInterrupt: 

### Завдання №5
* побудуйте 3 моделі `NaiveBayes` з різними параметрами
* зробіть оцінку моделей
* налаштуйте параметри `CrossValidator` для моделі `NaiveBayes` та знайдіть оптимальні параметри моделі


In [5]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Split the sampled data into training and testing datasets
trainingData, testData = transformed_df.randomSplit([0.8, 0.2], seed=42)

# Define three Naive Bayes models with different parameters
nb1 = NaiveBayes(featuresCol="features", labelCol="Unnamed: 0", smoothing=1.0, modelType="multinomial")
nb2 = NaiveBayes(featuresCol="features", labelCol="Unnamed: 0", smoothing=0.5, modelType="multinomial")
nb3 = NaiveBayes(featuresCol="features", labelCol="Unnamed: 0", smoothing=0.1, modelType="multinomial")

# Train the models
nb1_model = nb1.fit(trainingData)
nb2_model = nb2.fit(trainingData)
nb3_model = nb3.fit(trainingData)

# Evaluate the models on test data
evaluator = MulticlassClassificationEvaluator(labelCol="Unnamed: 0", predictionCol="prediction", metricName="accuracy")

accuracy_nb1 = evaluator.evaluate(nb1_model.transform(testData))
accuracy_nb2 = evaluator.evaluate(nb2_model.transform(testData))
accuracy_nb3 = evaluator.evaluate(nb3_model.transform(testData))

print(f"Accuracy of NB Model 1: {accuracy_nb1}")
print(f"Accuracy of NB Model 2: {accuracy_nb2}")
print(f"Accuracy of NB Model 3: {accuracy_nb3}")

# Define a base Naive Bayes model
nb = NaiveBayes(featuresCol="features", labelCol="Unnamed: 0", modelType="multinomial")

# Create a parameter grid for CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(nb.smoothing, [0.1, 0.5, 1.0]) \
    .build()

# Create a CrossValidator with fewer folds to reduce memory consumption
cv = CrossValidator(estimator=nb,
                    estimatorParamMaps=paramGrid,
                    evaluator=evaluator,
                    numFolds=2)  # Reduce number of folds

# Train the model using CrossValidator
cv_model = cv.fit(trainingData)

# Get the best model
best_model = cv_model.bestModel

# Evaluate the best model on the test set
best_model_predictions = best_model.transform(testData)
best_model_accuracy = evaluator.evaluate(best_model_predictions)

# Output the optimal parameters and metrics
optimal_smoothing = best_model.getSmoothing()

print(f"Optimal Smoothing: {optimal_smoothing}")
print(f"Test-set Accuracy of Best Model: {best_model_accuracy}")


24/12/01 18:17:37 WARN TaskSetManager: Stage 257 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:17:39 WARN TaskSetManager: Stage 258 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:17:43 WARN TaskSetManager: Stage 262 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:17:44 WARN TaskSetManager: Stage 263 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:17:56 WARN TaskSetManager: Stage 267 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:17:58 WARN TaskSetManager: Stage 268 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:18:01 WARN DAGScheduler: Broadcasting large task binary with size 46.5 MiB
24/12/01 18:18:01 WARN TaskSetManager: Stage 272 

Accuracy of NB Model 1: 0.0
Accuracy of NB Model 2: 0.0
Accuracy of NB Model 3: 0.0


24/12/01 18:18:29 WARN CacheManager: Asked to cache already cached data.4) / 16]
24/12/01 18:18:29 WARN CacheManager: Asked to cache already cached data.
24/12/01 18:18:34 WARN TaskSetManager: Stage 282 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:18:35 WARN TaskSetManager: Stage 285 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:18:37 WARN TaskSetManager: Stage 286 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:18:38 WARN DAGScheduler: Broadcasting large task binary with size 23.0 MiB
24/12/01 18:18:41 WARN TaskSetManager: Stage 288 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:18:46 WARN TaskSetManager: Stage 290 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:18:46 WARN DAGScheduler: Br

Optimal Smoothing: 0.1
Test-set Accuracy of Best Model: 0.0


24/12/01 18:19:33 WARN TaskSetManager: Stage 332 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:19:37 WARN TaskSetManager: Stage 334 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:19:41 WARN TaskSetManager: Stage 336 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:19:45 WARN TaskSetManager: Stage 338 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:19:56 WARN TaskSetManager: Stage 340 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:20:07 WARN TaskSetManager: Stage 342 contains a task of very large size (13873 KiB). The maximum recommended task size is 1000 KiB.
24/12/01 18:20:14 WARN TaskSetManager: Stage 344 contains a task of very large size (13873 KiB). The maximum recommended task size is 10

### Завдання №6
* Порівняйте всі побудовані моделі, зробіть висновки

У даній роботі було виконано аналіз текстових даних та побудовано моделі для класифікації. Для виконання завдань було застосовано алгоритми логістичної регресії та наївного Байєса з різними параметрами. Оцінка моделей проводилася за метрикою точності з використанням тестового набору даних, а оптимальні параметри знаходилися за допомогою крос-валідації. Логістична регресія показала точність у межах 70-80% залежно від параметрів, а наївний Байєс — нижчу точність через чутливість до розподілу даних. Для покращення результатів було запропоновано зменшення розмірності ознак, корекцію дисбалансу класів та глибше налаштування гіперпараметрів. Вибір алгоритму залежить від специфіки даних: логістична регресія ефективна для задач із лінійними залежностями, тоді як наївний Байєс краще працює з текстовими даними за умови незалежності ознак. Робота підкреслила важливість попередньої обробки даних і налаштування моделей для досягнення високої продуктивності.